In [47]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime
from datetime import date
import glob

from scipy.optimize import root_scalar
import scipy.optimize as opt

In [48]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [49]:
def find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file

def clean_position(position):
    position_copy = position.copy()
    position_copy = position_copy[
        position_copy["Current Value"].notna()
    ]  # remove rows without current value
    position_copy["Current Value"] = transfer_dollar_to_float(
        position_copy["Current Value"]
    )
    position_copy["Cost Basis Total"] = transfer_dollar_to_float(
        position_copy["Cost Basis Total"]
    )
    return position_copy

def transfer_dollar_to_float(dat):
    """
    Change "$123,456" to 123455
    """
    return dat.str.replace("$", "", regex=False).astype(float)


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)

    transactions = combine_transaction_files(transaction_files)
    transactions = clean_transactions(transactions)
    return transactions

def combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions


def clean_transactions(transactions):
    transactions_copy = transactions.copy()
    transactions_copy = transactions_copy[
        transactions_copy["Amount ($)"].notna()
    ]  # remove rows without  value
    transactions_copy["Run Date"] = pd.to_datetime(
        transactions_copy["Run Date"], format=" %m/%d/%Y"
    ).dt.date
    transactions_copy["Settlement Date"] = pd.to_datetime(
        transactions_copy["Settlement Date"], format="%m/%d/%Y"
    ).dt.date
    transactions_copy.loc[transactions_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    transactions_copy["Symbol"] = transactions_copy[
        "Symbol"
    ].str.lstrip()  # remove space at the beginning of Symbol
    transactions_copy = transactions_copy.sort_values(by="Run Date").reset_index(
        drop=True
    )
    return transactions_copy

In [50]:
## load position
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'


In [51]:
position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
position_files = glob.glob(position_file_path_pattern)
position_file = find_latest_position_file(position_files)
position = pd.read_csv(position_file)
position = clean_position(position)

In [52]:
transaction_file_path_pattern = os.path.join(
    data_folder_path, transaction_file_pattern
)
transaction_files = glob.glob(transaction_file_path_pattern)

transactions = combine_transaction_files(transaction_files)
transactions = clean_transactions(transactions)

In [79]:
class Portfolio:
    """
    A class to represent a fidelity portfolio.

    Attributes
    ----------
    transactions: pd.DataFrame
        All historial transactions.
    position: pd.DataFrame
        Current balance in the account.
    today: datetime.date
        Today's date
    individual_transactions: pd.DataFrame
        Transactions under account "Individual Z23390746"
    individual_position: pd.DataFrame
        Position under account "Individual Z23390746"

    Methods
    -------
    show_investment_distribution:
        display the result of get_investment_distribution.
    get_investment_distribution:
        get the distribution of investment amony differnet asset.
    get_total_investment:
        get total investment amount in dollar.
    get_stock_investment:
        get amount of stock investment in dollar.
    get_bill_investment:
        get amount of bill investment in dollar.
    get_other_investment:
        get amount of other investment in dollar.
    show_stock_irr:
        display the result of get_stock_irr.
    get_stock_irr:
        get the irr of each stock.
    add_total_current_value_to_individual_position:
        ---
    get_total_current_value:
        ---
    set_merged_individual_position_transaction:
        merge individual_position and individual_transaction.
    add_time_diff:
        ---
    calculate_irr(symbol_list, lower_bound=-0.999, upper_bound=5):
        filter all transactions with symbol in symbol_list. Then calculate the irr of these transactions.
    filter_merged_transactions_by_symbol(symbol_list):
        filter all transactions with symbol in symbol_list.
    """

    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position

        account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.today = datetime.now().date()
        self.individual_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["Individual"]
        ]
        self.pension_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["401k"]
        ]
        self.HSA_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["HSA"]
        ]
        self.cash_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["Cash"]
        ]
        
        self.individual_position = self.position[
            self.position["Account Number"] == account_number_dic["Individual"]
        ]
        self.pension_position = self.position[
            self.position["Account Number"] == account_number_dic["401k"]
        ]
        self.HSA_position = self.position[
            self.position["Account Number"] == account_number_dic["HSA"]
        ]
        self.cash_position = self.position[
            self.position["Account Number"] == account_number_dic["Cash"]
        ]
        

    def show_investment_distribution(self):
        investment_distribution = self.get_investment_distribution()
        investment_distribution["Percent"] = [
            f"{x * 100:.2f}%" for x in investment_distribution["Percent"]
        ]
        print(pd.DataFrame(investment_distribution))

    def get_investment_distribution(self):
        total_investment = self.get_total_investment()
        stock_investment = self.get_stock_investment()
        bill_investment = self.get_bill_investment()
        other_investment = self.get_other_investment()
        result = {
            "Class": ["stock", "bill", "other", "total"],
            "Amount": [
                stock_investment,
                bill_investment,
                other_investment,
                total_investment,
            ],
            "Percent": [
                stock_investment / total_investment,
                bill_investment / total_investment,
                other_investment / total_investment,
                total_investment / total_investment,
            ],
        }

        return result

    def get_total_investment(self):
        total_investment = self.individual_transactions[
            self.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"].sum()
        return total_investment

    def get_stock_investment(self):
        stock_position = self.individual_position[
            ~(self.individual_position["Description"] == "HELD IN MONEY MARKET")
            & ~(self.individual_position["Description"].str.contains("BILLS", na=False))
        ]
        return stock_position["Cost Basis Total"].sum()

    def get_bill_investment(self):
        bill_position = self.individual_position[
            (self.individual_position["Description"].str.contains("BILLS", na=False))
        ]
        return bill_position["Cost Basis Total"].sum()

    def get_other_investment(self):
        return (
            self.get_total_investment()
            - self.get_stock_investment()
            - self.get_bill_investment()
        )

    def show_stock_irr(self):
        stock_irr = self.get_stock_irr()
        stock_irr_nice_look = pd.DataFrame(
            {
                "Stock": stock_irr.keys(),
                "irr": [f"{(stock_irr[x]) * 100:.2f}%" for x in stock_irr.keys()],
            }
        )
        print(pd.DataFrame(stock_irr_nice_look))

    def get_stock_irr(self):
        self.add_total_current_value_to_individual_position()
        self.set_merged_individual_position_transaction()
        self.merged_individual_position_transaction = self.add_time_diff(self.merged_individual_position_transaction)

        unique_symbols = self.merged_individual_position_transaction["Symbol"].unique()
        stock_list = [
            element
            for element in unique_symbols
            if "FZFXX" not in element
            and not element[0].isdigit()
            and "Pending" not in element
        ]
        result_dict = {x: self.calculate_irr(self.filter_merged_transactions_by_symbol([x])) for x in stock_list}
        if "Transfer" in stock_list:
            stock_list.remove("Transfer")
        transactions = self.filter_merged_transactions_by_symbol(stock_list)
        result_dict["stock"] = self.calculate_irr(transactions)

        return result_dict

    def add_total_current_value_to_individual_position(self):
        if "Transfer" in self.individual_position["Symbol"].values:
            print("Total current value has been added")
            return

        total_current_value = self.get_total_current_value()
        new_rows = pd.DataFrame(
            {"Symbol": ["Transfer"], "Current Value": -total_current_value}
        )
        self.individual_position = pd.concat(
            [self.individual_position, new_rows], ignore_index=True
        )

    def get_total_current_value(self):
        return self.individual_position["Current Value"].sum()

    def set_merged_individual_position_transaction(self):
        new_rows = pd.DataFrame(
            {
                "Run Date": [self.today] * len(self.individual_position),
                "Symbol": self.individual_position["Symbol"],
                "Amount ($)": self.individual_position["Current Value"],
            }
        )

        self.merged_individual_position_transaction = pd.concat(
            [self.individual_transactions, new_rows], ignore_index=True
        )

    def add_time_diff(self, transactions):
        transactions_copy = transactions.copy()
        if not 'time_diffs_in_year' in transactions_copy.columns:
            transactions_copy.loc[:, "time_diffs_in_year"] = (
                self.today - transactions_copy.loc[:, "Run Date"]
            ).apply(lambda x: x.days) / 365.25
        else:
            print("Column time_diffs_in_year exists")
        return transactions_copy
        

    def calculate_irr(self, transactions, lower_bound=-0.999, upper_bound=5):

        def npv(rate):
            return np.sum(
                transactions["Amount ($)"]
                * (1 + rate) ** transactions["time_diffs_in_year"]
            )

        result = root_scalar(npv, bracket=[lower_bound, upper_bound], method="bisect")

        if result.converged:
            return result.root
        else:
            raise ValueError(
                "No solution found for the rate that satisfies the equation."
            )

    def filter_merged_transactions_by_symbol(self, symbol_list):
        return self.merged_individual_position_transaction[
            self.merged_individual_position_transaction["Symbol"].isin(symbol_list)
        ]
        
    

In [80]:
current_portfolio = Portfolio(transactions=transactions, position=position)

In [82]:
current_portfolio.transactions[
            current_portfolio.transactions["Account"] == "Individual"
        ]

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,2022-07-26,Individual,Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.00,NaN,NaN,NaN,NaN,1000.00,NaT
1,2022-07-29,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT
2,2022-07-29,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.00,NaN,NaN,NaN,-0.22,NaT
3,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.00,163.54,NaN,NaN,NaN,-163.54,2022-08-09
4,2022-08-05,Individual,Z23390746,YOU BOUGHT STARBUCKS CORP COM USD0.001 (SBUX)...,SBUX,STARBUCKS CORP COM USD0.001,Cash,9.00,85.36,NaN,NaN,NaN,-768.20,2022-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2025-03-31,Individual,Z23390746,DIVIDEND RECEIVED PEPSICO INC (PEP) (Cash),PEP,PEPSICO INC,Cash,0.00,NaN,NaN,NaN,NaN,13.55,NaT
476,2025-03-31,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,441.08,1.00,NaN,NaN,NaN,-441.08,NaT
477,2025-03-31,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,441.08,NaT
480,2025-04-01,Individual,Z23390746,DIVIDEND RECEIVED NIKE INC CLASS B COM NPV (N...,NKE,NIKE INC CLASS B COM NPV,Cash,0.00,NaN,NaN,NaN,NaN,12.00,NaT


In [83]:
current_portfolio.individual_transactions[
            current_portfolio.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"]

0        1000.0
7       29000.0
91      30000.0
100     40000.0
142      1500.0
161    148500.0
162   -148500.0
164    148500.0
171    100000.0
175   -100000.0
180    100000.0
181   -100000.0
184    140000.0
190     50000.0
195     10000.0
211     50000.0
269     50000.0
302     50000.0
368    150000.0
412     50000.0
415     50000.0
420     50000.0
439     50000.0
452     25500.0
454     24500.0
458     50000.0
Name: Amount ($), dtype: float64

In [84]:
current_portfolio.get_total_investment()

1050000.0

In [85]:
[current_portfolio.get_stock_investment(),current_portfolio.get_bill_investment(),current_portfolio.get_other_investment()]

[311697.68999999994, 754548.9500000001, -16246.640000000014]

In [86]:
current_portfolio.get_investment_distribution()

{'Class': ['stock', 'bill', 'other', 'total'],
 'Amount': [311697.68999999994,
  754548.9500000001,
  -16246.640000000014,
  1050000.0],
 'Percent': [0.2968549428571428,
  0.7186180476190477,
  -0.015472990476190489,
  1.0]}

In [88]:
current_portfolio.show_investment_distribution()

   Class      Amount  Percent
0  stock   311697.69   29.69%
1   bill   754548.95   71.86%
2  other   -16246.64   -1.55%
3  total  1050000.00  100.00%


In [89]:
current_portfolio.show_stock_irr()

       Stock      irr
0   Transfer    5.92%
1       AAPL   16.49%
2       SBUX   11.79%
3        JPM   34.75%
4        AXP   27.34%
5       AMZN   39.83%
6      GOOGL   29.38%
7        NKE  -18.42%
8       TSLA   33.93%
9      FXAIX    5.04%
10       MCD   27.43%
11     FSPSX    6.94%
12     FSKAX    0.57%
13      COKE   39.16%
14      MSFT  -18.44%
15      BRKB   39.39%
16       KHC   -8.48%
17        VZ   33.02%
18      NVDA  -34.22%
19       PEP   37.53%
20     stock    7.91%
